In [10]:
import numpy as np 
from tqdm import tqdm 
#from mySSA import mySSA
import matplotlib.pyplot as plt 
import pandas as pd 
import warnings
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

import logging
logging.getLogger('fbprophet').setLevel(logging.ERROR)


In [11]:
root_folder = '/home/alqua/data/data_vdfs/'

df = pd.read_parquet(root_folder + "sym_data/sym_kf_1h_with_inflow_and_rain_forecast.parquet")
df = df.set_index("time")
df = df[["inflow_kf", "precip"]]


In [12]:
start_date = pd.to_datetime('2023-05-01 00:00:00+00:00')
date_range = pd.date_range(start=start_date, periods=len(df), freq='H')
df.index = date_range

/tmp/ipykernel_82554/3343584635.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start=start_date, periods=len(df), freq='H')


In [13]:
from scipy import stats
from prophet import Prophet

df["inflow_kf"], _ = stats.boxcox(df["inflow_kf"].values)
df = df.reset_index()
df = df.rename(columns={'index':'ds', 'inflow_kf':'y'})
df['ds'] = df['ds'].dt.tz_localize(None)

holiday_df = df[df['precip'] > 0][['ds']]
holiday_df['holiday'] = 'rainy_day'
holiday_df['lower_window'] = 0
holiday_df['upper_window'] = 1

In [14]:
df = df.dropna()

In [15]:
m = Prophet(daily_seasonality=True)
m.fit(df[:240])
future = m.make_future_dataframe(periods=24, freq='H', include_history=False)
forecast = m.predict(future)

09:09:54 - cmdstanpy - INFO - Chain [1] start processing
09:09:54 - cmdstanpy - INFO - Chain [1] done processing
/home/alqua/anaconda3/envs/opt_env/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(


In [16]:
window_size = 500
steps = 400
ytrue = []
predictions = []
L = 200
K = window_size - L + 1
steps = 24
streams10 = [i for i in range(3)]
 
for i in tqdm(range(0, len(df) - window_size + 1, steps)):
    ts_slice = df.iloc[i:i + window_size, :]
    m = Prophet(daily_seasonality=True, 
                
                seasonality_mode='multiplicative',
                holidays = holiday_df,
                changepoint_prior_scale=0.001)
    m.add_regressor('precip', prior_scale=2)
    m.add_seasonality(name='hourly', period=5/24, fourier_order=5)
    m.fit(ts_slice)
    future = m.make_future_dataframe(periods=24, freq='H', include_history=False)
    future["precip"] = df.iloc[i+window_size:i+window_size+24, 2][:24].values
    future["rain"] =  np.where(df.iloc[i+window_size:i+window_size+24, 2][:24].values > 0, 1, 0)
    forecast = m.predict(future)
    forecast["ytrue"] = df.iloc[i+window_size:i+window_size+24, 1:2].values.flatten()
    predictions.append(forecast)
    if i == 24:
        break
    

  0%|          | 0/156 [00:00<?, ?it/s]09:09:54 - cmdstanpy - INFO - Chain [1] start processing
09:09:54 - cmdstanpy - INFO - Chain [1] done processing
/home/alqua/anaconda3/envs/opt_env/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(
  1%|          | 1/156 [00:00<01:23,  1.85it/s]09:09:55 - cmdstanpy - INFO - Chain [1] start processing
09:09:55 - cmdstanpy - INFO - Chain [1] done processing
/home/alqua/anaconda3/envs/opt_env/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(
  1%|          | 1/156 [00:01<02:45,  1.07s/it]


In [17]:
results_df = pd.concat(predictions)

In [18]:
results_df

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,daily,daily_lower,daily_upper,extra_regressors_multiplicative,...,rainy_day_lower,rainy_day_upper,weekly,weekly_lower,weekly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat,ytrue
0,2023-05-21 20:00:00,2.078041,2.081626,2.102250,2.078041,2.078041,0.002993,0.002993,0.002993,-0.000023,...,0.0,0.0,-0.003126,-0.003126,-0.003126,0.0,0.0,0.0,2.091969,2.089467
1,2023-05-21 21:00:00,2.077957,2.080068,2.099866,2.077957,2.077957,0.001885,0.001885,0.001885,-0.000023,...,0.0,0.0,-0.002991,-0.002991,-0.002991,0.0,0.0,0.0,2.090022,2.085871
2,2023-05-21 22:00:00,2.077872,2.077137,2.097587,2.077872,2.077872,0.000576,0.000576,0.000576,-0.000023,...,0.0,0.0,-0.002854,-0.002854,-0.002854,0.0,0.0,0.0,2.087547,2.082624
3,2023-05-21 23:00:00,2.077787,2.075132,2.094540,2.077787,2.077787,-0.001062,-0.001062,-0.001062,-0.000023,...,0.0,0.0,-0.002716,-0.002716,-0.002716,0.0,0.0,0.0,2.084519,2.077346
4,2023-05-22 00:00:00,2.077703,2.069680,2.090635,2.077702,2.077704,-0.003204,-0.003204,-0.003204,-0.000023,...,0.0,0.0,-0.002577,-0.002577,-0.002577,0.0,0.0,0.0,2.080066,2.072805
5,2023-05-22 01:00:00,2.077618,2.064851,2.085013,2.077616,2.077621,-0.005778,-0.005778,-0.005778,-0.000023,...,0.0,0.0,-0.002437,-0.002437,-0.002437,0.0,0.0,0.0,2.074752,2.067468
6,2023-05-22 02:00:00,2.077533,2.058815,2.080205,2.077530,2.077539,-0.008278,-0.008278,-0.008278,-0.000023,...,0.0,0.0,-0.002296,-0.002296,-0.002296,0.0,0.0,0.0,2.069927,2.063186
7,2023-05-22 03:00:00,2.077449,2.056943,2.077277,2.077443,2.077457,-0.009891,-0.009891,-0.009891,-0.000023,...,0.0,0.0,-0.002155,-0.002155,-0.002155,0.0,0.0,0.0,2.066828,2.058816
8,2023-05-22 04:00:00,2.077364,2.057634,2.077437,2.077356,2.077375,-0.009928,-0.009928,-0.009928,-0.000023,...,0.0,0.0,-0.002014,-0.002014,-0.002014,0.0,0.0,0.0,2.067136,2.057698
9,2023-05-22 05:00:00,2.077279,2.060641,2.080504,2.077269,2.077294,-0.008255,-0.008255,-0.008255,-0.000023,...,0.0,0.0,-0.001872,-0.001872,-0.001872,0.0,0.0,0.0,2.070613,2.060907


In [19]:
results_df = results_df.reset_index(drop=True)

In [20]:
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler
 
 
fig = FigureWidgetResampler(go.Figure())
fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))
fig.add_trace(go.Scattergl(name=r'Forecast', showlegend=True), hf_x=results_df["ds"], hf_y=results_df['yhat'])
fig.add_trace(go.Scattergl(name=r'True', showlegend=True), hf_x=results_df["ds"], hf_y=results_df['ytrue'])
fig.update_layout(height=400, template="plotly_dark")
display(fig)

FigureWidgetResampler({
    'data': [{'name': 'Forecast',
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'c8fc916c-aef5-4e17-9342-4cf20a597f1a',
              'x': array([datetime.datetime(2023, 5, 21, 20, 0),
                          datetime.datetime(2023, 5, 21, 21, 0),
                          datetime.datetime(2023, 5, 21, 22, 0),
                          datetime.datetime(2023, 5, 21, 23, 0),
                          datetime.datetime(2023, 5, 22, 0, 0),
                          datetime.datetime(2023, 5, 22, 1, 0),
                          datetime.datetime(2023, 5, 22, 2, 0),
                          datetime.datetime(2023, 5, 22, 3, 0),
                          datetime.datetime(2023, 5, 22, 4, 0),
                          datetime.datetime(2023, 5, 22, 5, 0),
                          datetime.datetime(2023, 5, 22, 6, 0),
                          datetime.datetime(2023, 5, 22, 7, 0),
                          datetime.datet

In [21]:
holiday_df

,ds,holiday,lower_window,upper_window
84,2023-05-04 12:00:00,rainy_day,0,1
85,2023-05-04 13:00:00,rainy_day,0,1
86,2023-05-04 14:00:00,rainy_day,0,1
87,2023-05-04 15:00:00,rainy_day,0,1
88,2023-05-04 16:00:00,rainy_day,0,1
...,...,...,...,...
4218,2023-10-23 18:00:00,rainy_day,0,1
4220,2023-10-23 20:00:00,rainy_day,0,1
4221,2023-10-23 21:00:00,rainy_day,0,1
4223,2023-10-23 23:00:00,rainy_day,0,1
